# Tugas Akhir STBI
## "Healthcare Chatbot for Assistance Using QA Model with Word Embedding-Based Information Retrieval"

Anggota Kelompok:
* Rhafael Chandra (22/498550/PA/21528)
* Nathanael Aurelino Sulistyo (22/497480/PA/21422)
* Rahmad Ramadhan (22/494516/PA/21278)

In [1]:
import pandas as pd

import numpy as np
import scipy as sp
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm.notebook import tqdm
import time
from nltk.translate.chrf_score import sentence_chrf
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
import nltk
import textwrap

In [2]:
import gensim
import sklearn

## Load Gemini API

In [3]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
load_dotenv()

GEMINI_API_KEY =  os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)
model_llm = genai.GenerativeModel("gemini-1.5-flash")

## Load Dataset

In [4]:
df_medquad = pd.read_csv("MedQuad-MedicalQnADataset.csv")
df_medquad.head()

,Unnamed: 0,qtype,Question,Answer
0,0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


In [5]:
df_pubmed_train = pd.read_csv("PubMed_200k_RCT/train.csv")
df_pubmed_test = pd.read_csv("PubMed_200k_RCT/test.csv")
df_pubmed = pd.concat([df_pubmed_train, df_pubmed_test], ignore_index=True)
df_pubmed.head()

,abstract_id,line_id,abstract_text,line_number,total_lines,target
0,24491034,24491034_0_11,The emergence of HIV as a chronic condition me...,0,11,BACKGROUND
1,24491034,24491034_1_11,This paper describes the design and evaluation...,1,11,BACKGROUND
2,24491034,24491034_2_11,This study is designed as a randomised control...,2,11,METHODS
3,24491034,24491034_3_11,The intervention group will participate in the...,3,11,METHODS
4,24491034,24491034_4_11,The program is based on self-efficacy theory a...,4,11,METHODS


# Preprocessing

In [6]:
df_pubmed = df_pubmed.dropna(subset=['abstract_text'])  # Remove rows with NaN in abstract_text
df_pubmed = df_pubmed.sort_values(by=['line_id'])
df_pubmed = df_pubmed.groupby('abstract_id').agg({'abstract_text':' '.join}).reset_index()
df_pubmed.head()

,abstract_id,abstract_text
0,1279170,We conducted this study to assess the clinical...
1,1281030,To determine whether prophylactic treatment wi...
2,1282364,After the discovery of type C hepatitis virus ...
3,1283117,Since it is not clear whether testosterone or ...
4,1283730,The aim was to study the pharmacokinetic param...


In [7]:
# add qa id to medquad
df_medquad['qa_id'] = range(1, len(df_medquad) + 1)
df_medquad = df_medquad[['qa_id', 'Question', 'Answer', 'qtype']]
df_medquad.head()

,qa_id,Question,Answer,qtype
0,1,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...,susceptibility
1,2,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...,symptoms
2,3,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...,susceptibility
3,4,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos...",exams and tests
4,5,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen...",treatment


In [8]:
train_medquad, test_medquad = train_test_split(df_medquad, test_size=0.2, random_state=42)

# Information Retrieval Model 2

## Word Embedding: BERT

In [9]:
# Load BioBERT
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")

def get_biobert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, 
                      truncation=True, padding=True)
    
    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    
    return embeddings

def get_sentence_vector(sentence):
    return get_biobert_embedding(sentence)

In [10]:
def process_embeddings(df):
    print("Generating embeddings...")
    
    # Process abstract texts
    abstract_vectors = []
    for text in tqdm(df['abstract_text'], desc="Processing abstracts"):
        abstract_vectors.append(get_biobert_embedding(text))
    df['abstract_vector'] = abstract_vectors
    
    # Process questions if they exist
    if 'Question' in df.columns:
        question_vectors = []
        for text in tqdm(df['Question'], desc="Processing questions"):
            question_vectors.append(get_biobert_embedding(text))
        df['question_vector'] = question_vectors
    
    return df

### Load Pubmed Embeddings

* Pubmed Embeddings which is processed by Biobert is made in Kaggle for faster processing speed
* The embeddings are processed by Biobert and saved in 8 different files
* The files are then loaded and combined into a single dataframe
* The embeddings are then converted back to numpy arrays

In [11]:
def load_and_convert_embeddings(file_num):
    df = pd.read_csv(f'bert_processed_embeddings/bert_processed_embeddings_{file_num}.csv')
    # Convert string representation of vectors back to numpy arrays
    df['abstract_vector'] = df['abstract_vector'].apply(
        lambda x: np.array([float(i) for i in x.split(',')])
    )
    return df

def combine_all_embeddings():
    combined_df = load_and_convert_embeddings(1)
    
    # Load and concatenate remaining files
    for i in range(2, 9):
        try:
            df = load_and_convert_embeddings(i)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            print(f"Loaded and combined embeddings_{i}")
        except FileNotFoundError:
            print(f"Warning: processed_embeddings_{i}.csv not found")
    
    return combined_df

# Load and combine all embeddings
pubmed_embeddings = combine_all_embeddings()

print(f"Total number of embeddings: {len(pubmed_embeddings)}")
print(f"Shape of first embedding vector: {pubmed_embeddings['abstract_vector'].iloc[0].shape}")
print("Type of first embedding vector:", type(pubmed_embeddings['abstract_vector'].iloc[0]))

Loaded and combined embeddings_2


model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Loaded and combined embeddings_3
Loaded and combined embeddings_4
Loaded and combined embeddings_5
Loaded and combined embeddings_6
Loaded and combined embeddings_7
Loaded and combined embeddings_8
Total number of embeddings: 193154
Shape of first embedding vector: (768,)
Type of first embedding vector: <class 'numpy.ndarray'>


In [12]:
pubmed_embeddings.head()

,abstract_id,abstract_text,abstract_vector
0,1279170,We conducted this study to assess the clinical...,"[-0.5401623, -0.018637111, 0.27044067, -0.0284..."
1,1281030,To determine whether prophylactic treatment wi...,"[-0.39741406, -0.06910312, 0.2548711, -0.20528..."
2,1282364,After the discovery of type C hepatitis virus ...,"[-0.4540933, -0.008601877, 0.027230231, -0.226..."
3,1283117,Since it is not clear whether testosterone or ...,"[-0.40724534, 0.07242108, 0.096013926, -0.0294..."
4,1283730,The aim was to study the pharmacokinetic param...,"[-0.48465633, -0.058801036, 0.3869856, -0.0874..."


### Load Medquad Embeddings

* Medquad Embeddings which is processed by Biobert is made in Kaggle for faster processing speed
* The embeddings are processed by Biobert and saved in 8 different files
* The files are then loaded and combined into a single dataframe
* The embeddings are then converted back to numpy arrays

In [13]:
def extract_vector(vector_str):
    # Remove brackets and split by spaces
    vector_str = vector_str.strip('[]')
    # Convert to numpy array of floats
    return np.array([float(x) for x in vector_str.split()])

medquad_embeddings = pd.read_csv(f'biobert_processed_embeddings/train_medquad.csv')
medquad_embeddings['question_vector'] = medquad_embeddings['question_vector'].apply(extract_vector)

print(f"Total number of embeddings: {len(medquad_embeddings)}")
print(f"Shape of first embedding vector: {medquad_embeddings['question_vector'].iloc[0].shape}")
print("Type of first embedding vector:", type(medquad_embeddings['question_vector'].iloc[0]))

Total number of embeddings: 13125
Shape of first embedding vector: (768,)
Type of first embedding vector: <class 'numpy.ndarray'>


### Retrieval Functions

In [14]:
def find_top_k_abstracts(df, query_vector, abstract_vectors_col, k=3):
    cosine_scores = cosine_similarity([query_vector], np.array(df[abstract_vectors_col]).tolist())[0]
    top_k_indices = np.argsort(cosine_scores)[-k:][::-1]
    return df.iloc[top_k_indices]['abstract_id'].values, cosine_scores[top_k_indices]

def find_top_k_answers(df, query_vector, question_vectors_col, k=3):
    cosine_scores = cosine_similarity([query_vector], np.array(df[question_vectors_col]).tolist())[0]
    top_k_indices = np.argsort(cosine_scores)[-k:][::-1]
    return df.iloc[top_k_indices]['qa_id'].values, cosine_scores[top_k_indices]

def get_qa_pairs(df, qa_ids):
    return df[df['qa_id'].isin(qa_ids)][['qa_id', 'Question', 'Answer']]

def get_abstracts(df, abstract_ids):
    return df[df['abstract_id'].isin(abstract_ids)][['abstract_id', 'abstract_text']]

def query_top_k_answers_and_abstracts(df_1, df_2, query, k=10):
    query_vector = get_sentence_vector(query)

    qa_ids, question_scores = find_top_k_answers(df_1, query_vector, 'question_vector', k)
    abstract_ids, abstract_scores = find_top_k_abstracts(df_2, query_vector, 'abstract_vector', k)

    answers_df = get_qa_pairs(df_1, qa_ids)
    abstracts_df = get_abstracts(df_2, abstract_ids)

    answers_df['Similarity Score'] = question_scores
    abstracts_df['Similarity Score'] = abstract_scores

    return answers_df, abstracts_df

In [15]:
queries = [
    "What is the incubation period of COVID-19?",
    "Cure for fever?",
    "Who is at risk for Lymphocytic Choriomeningit",
    "What are the symptoms of Ligma?"
]

In [16]:
for query in queries:
    print(f"Query: {query}")
    answers_df, abstracts_df = query_top_k_answers_and_abstracts(medquad_embeddings, pubmed_embeddings, query)
    print("Top Answers:")
    display(answers_df)
    print("Top Abstracts:")
    display(abstracts_df)
    print("\n")

Query: What is the incubation period of COVID-19?
Top Answers:


,qa_id,Question,Answer,Similarity Score
658,14465,What are the symptoms of Fucosidosis type 1 ?,What are the signs and symptoms of Fucosidosis...,0.911123
711,12893,What are the symptoms of Cronkhite-Canada dise...,What are the signs and symptoms of Cronkhite-C...,0.910592
888,115,Who is at risk for Nocardiosis? ?,People with very weak immune (body defense) sy...,0.910592
1929,11019,What is (are) Zika virus infection ?,Zika virus infection is spread to people prima...,0.909483
2831,11020,What are the treatments for Zika virus infecti...,How might a Zika virus infection be treated? T...,0.909286
5733,157,Who is at risk for Hendra Virus Disease (HeV)? ?,Transmission of Hendra virus to humans can occ...,0.907044
5844,5270,What is (are) Zika Virus ?,Zika is a virus that is spread by mosquitoes. ...,0.906939
8197,153,what is the history of hps for Hantavirus ?,"The ""First""Outbreak\n \nIn May 1993, an outbr...",0.905759
10660,159,Who is at risk for Hendra Virus Disease (HeV)? ?,Australia’s “Flying fox” bats (genus Pteropus)...,0.904658
11985,12892,What is (are) Cronkhite-Canada disease ?,Cronkhite-Canada syndrome is a rare gastrointe...,0.904617


Top Abstracts:


,abstract_id,abstract_text,Similarity Score
9691,8792879,It was hypothesized that non-depolarizing neur...,0.010274
14882,9403144,Previous research investigating the interperso...,0.009608
19806,9871866,To analyze the frequency distribution and desc...,0.008725
58416,15342159,To assess the effects of visual feedback gain ...,0.008593
113809,19832962,The ability to optically section live biologic...,0.008133
120987,20464814,Visual performance in dim light is compromised...,0.006956
154368,22957771,The aim was to evaluate the visual performance...,0.006566
157335,23160438,The magnification produced by a low-vision tel...,0.006410
160010,23337440,"Brains , like other physical devices , are inh...",0.005776
189609,25518099,This article presents the results of research ...,0.005438




Query: Cure for fever?
Top Answers:


...

,abstract_id,abstract_text,Similarity Score
19806,9871866,To analyze the frequency distribution and desc...,0.023877
101420,18833582,The aim of this pilot study was to examine whe...,0.022978
111424,19653485,The aim of this study was to assess the voice ...,0.022826
111425,19653486,Our aim was to investigate how the congruency ...,0.022091
113809,19832962,The ability to optically section live biologic...,0.020673
132203,21387017,Numerosity estimation is a basic preverbal abi...,0.020341
135250,21617516,The objective of this study was to assess the ...,0.019571
154368,22957771,The aim was to evaluate the visual performance...,0.019150
157335,23160438,The magnification produced by a low-vision tel...,0.018455
160010,23337440,"Brains , like other physical devices , are inh...",0.017716




Query: What are the symptoms of Ligma?
Top Answers:


,qa_id,Question,Answer,Similarity Score
1588,15793,What are the symptoms of Tucker syndrome ?,What are the signs and symptoms of Tucker synd...,0.970509
2327,12428,What are the symptoms of C syndrome ?,What are the signs and symptoms of C syndrome?...,0.968633
2738,15212,What are the symptoms of Aniridia ?,What are the signs and symptoms of Aniridia? T...,0.967809
3722,14816,What are the symptoms of Reynolds syndrome ?,What are the signs and symptoms of Reynolds sy...,0.965991
4053,12924,What are the symptoms of Young syndrome ?,What are the signs and symptoms of Young syndr...,0.965253
6209,14996,What are the symptoms of Fountain syndrome ?,What are the signs and symptoms of Fountain sy...,0.965004
6596,15226,What are the symptoms of Miller syndrome ?,What are the signs and symptoms of Miller synd...,0.964801
6897,12855,What are the symptoms of N syndrome ?,What are the signs and symptoms of N syndrome?...,0.964360
11311,12386,What are the symptoms of Ainhum ?,What are the signs and symptoms of Ainhum? The...,0.964298
11406,15904,What are the symptoms of Arts syndrome ?,What are the signs and symptoms of Arts syndro...,0.964258


Top Abstracts:


,abstract_id,abstract_text,Similarity Score
14882,9403144,Previous research investigating the interperso...,0.002990
19806,9871866,To analyze the frequency distribution and desc...,0.001858
111424,19653485,The aim of this study was to assess the voice ...,0.001679
111425,19653486,Our aim was to investigate how the congruency ...,0.001620
113809,19832962,The ability to optically section live biologic...,0.001538
138228,21830512,Directional sounds simulated using nonindividu...,0.001121
148127,22502713,This study was an attempt to identify associat...,-0.000732
155659,23046005,The purpose of this study was to assess patien...,-0.001601
160010,23337440,"Brains , like other physical devices , are inh...",-0.002399
186845,25257294,This research used context aware experiential ...,-0.002902


# Pass Context to LLM

In [17]:
def call_llm(context, query):
    prompt = f"You are a medical expert assistant. Answer the following medical question comprehensively and accurately. If the provided context contains relevant information, use it. If not, use your general medical knowledge to provide the best possible answer."

    prompt += f"\nQuestion: {query}\n"

    prompt += "\nContext:\n"
    prompt += "\nRelevant Medical QA Pairs:\n"
    for i, qa in enumerate(context['answers'], 1):
        prompt += f"\nQA Pair {i}:\nQ: {qa['Question']}\nA: {qa['Answer']}\n"
    
    prompt += "\nRelevant Medical Research Abstracts:\n"
    for i, abstract in enumerate(context['abstracts'], 1):
        prompt += f"\nAbstract {i}:\n{abstract['abstract_text']}\n"
    
    prompt += "\nProvide a clear, direct, and comprehensive answer to the question. Focus on being helpful and informative to the user."

    time.sleep(2)
    response = model_llm.generate_content(prompt)
    
    return response 

In [18]:
def process_medical_query(df_1, df_2, query):
    answers_df, abstracts_df = query_top_k_answers_and_abstracts(df_1, df_2, query)
    
    if len(answers_df) == 0 and len(abstracts_df) == 0:
        return {
            'response': "I apologize, but I don't have enough reliable information to answer this question.",
            'confidence': 'low'
        }
    
    context = {
        'answers': answers_df.to_dict(orient='records'),
        'abstracts': abstracts_df.to_dict(orient='records')
    }
    
    llm_response = call_llm(context, query)
    
    return {
        'response': llm_response.text,
        'confidence': 'high'
    }

test_query = "Who is at risk for Lymphocytic Choriomeningit?"
result = process_medical_query(medquad_embeddings, pubmed_embeddings, test_query)
print(result['response'])

Individuals at risk for Lymphocytic Choriomeningitis (LCM) include those who come into contact with the urine, feces, saliva, or blood of wild mice.  Pet mice and hamsters can also carry the virus, particularly if they originate from contaminated colonies or have contact with wild mice.  Human fetuses are at risk of infection from an infected mother (vertical transmission).  Laboratory workers who handle infected animals or the virus itself are also at risk, although this risk can be mitigated through proper safety precautions and sourcing animals from regularly tested colonies.



# Evaluation

## Test on Medquad Test Dataset

In [25]:
def normalize_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

def truncate_or_pad_text(text, target_length):
    words = text.split()
    if len(words) > target_length:
        # Truncate to target length
        return ' '.join(words[:target_length])
    return text

def calculate_rouge(prediction, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(prediction, reference)
    return {
        'ROUGE-1': scores['rouge1'].fmeasure,
        'ROUGE-L': scores['rougeL'].fmeasure
    }

def calculate_meteor(prediction, reference):
    pred_tokens = nltk.word_tokenize(prediction.lower())
    ref_tokens = nltk.word_tokenize(reference.lower())
    return meteor_score([ref_tokens], pred_tokens)

def calculate_chrf(prediction, reference):
    return sentence_chrf(reference, prediction, min_len=1, beta=2.0)

def calculate_metrics(predictions, references):
    individual_scores = []
    
    for pred, ref in zip(predictions, references):
        # Normalize texts to lowercase and remove extra whitespace
        pred = ' '.join(pred.lower().split())
        ref = ' '.join(ref.lower().split())
        
        # Calculate length ratio for normalization
        pred_len = len(pred.split())
        ref_len = len(ref.split())
        # Option 1: Use min(pred_len, ref_len) / max(pred_len, ref_len)
        ratio = pred_len / ref_len if pred_len > 0 else 1
        
        scores = {}
        
        # Calculate ROUGE scores and normalize
        rouge_scores = calculate_rouge(pred, ref)
        scores['ROUGE-1'] = rouge_scores['ROUGE-1'] * ratio
        scores['ROUGE-L'] = rouge_scores['ROUGE-L'] * ratio
        
        # Calculate and normalize METEOR score
        scores['METEOR'] = calculate_meteor(pred, ref) * ratio
        
        # Calculate and normalize chrF2 score
        scores['chrF2'] = calculate_chrf(pred, ref) * ratio
        
        individual_scores.append(scores)
    
    # Calculate average scores
    avg_scores = {
        metric: np.mean([s[metric] for s in individual_scores])
        for metric in individual_scores[0].keys()
    }
    
    return avg_scores, individual_scores

def evaluate_llm(test_df, medquad_embeddings, pubmed_embeddings):
    # Define specific test questions
    test_questions = [
    "What is (are) lactate dehydrogenase deficiency ?",
    "Is 21-hydroxylase deficiency inherited ?",
    "Is Loeys-Dietz syndrome inherited ?",
    "What are the genetic changes related to cerebral cavernous malformation ?"
    ]
    
    # Get corresponding true answers from test_df
    test_sample = test_df[test_df['Question'].isin(test_questions)]
    
    questions = test_sample['Question'].tolist()
    true_answers = test_sample['Answer'].tolist()
    
    # Get predictions
    predicted_answers = []
    for question in questions:
        result = process_medical_query(medquad_embeddings, pubmed_embeddings, question)
        predicted_answers.append(str(result['response']))
    
    # Ensure we have predictions
    if len(predicted_answers) == 0:
        raise ValueError("No predictions were generated")
    
    # Calculate all metrics
    avg_metrics, individual_metrics = calculate_metrics(predicted_answers, true_answers)
    
    # Create evaluation dataframe
    eval_data = []
    
    for i, (question, true_ans, pred_ans, metrics) in enumerate(zip(
        questions, true_answers, predicted_answers, individual_metrics)):
        
        true_ans_norm = normalize_text(true_ans)
        pred_ans_norm = truncate_or_pad_text(
            normalize_text(pred_ans), 
            len(true_ans_norm.split())
        )
        
        eval_data.append({
            'Question': question,
            'True Answer': true_ans,
            'Predicted Answer': pred_ans,
            'Normalized True Answer': true_ans_norm,
            'Normalized Predicted Answer': pred_ans_norm,
            'Original True Length': len(true_ans.split()),
            'Original Predicted Length': len(pred_ans.split()),
            'Normalized Length': len(true_ans_norm.split()),
            **metrics
        })
    
    eval_df = pd.DataFrame(eval_data)
    
    print("\nDetailed Evaluation Results:")
    print("\nMetrics Summary:")
    metrics_cols = ['ROUGE-1', 'ROUGE-L', 'METEOR', 'chrF2']
    print(eval_df[metrics_cols].describe())
    
    print("\nLength Analysis:")
    print(f"Average Original True Length: {eval_df['Original True Length'].mean():.1f} words")
    print(f"Average Original Predicted Length: {eval_df['Original Predicted Length'].mean():.1f} words")
    print(f"Average Normalized Length: {eval_df['Normalized Length'].mean():.1f} words")
    
    return eval_df, avg_metrics

# Run evaluation
print("Starting evaluation...")
eval_df, avg_metrics = evaluate_llm(test_medquad, medquad_embeddings, pubmed_embeddings)


Starting evaluation...

Detailed Evaluation Results:

Metrics Summary:
        ROUGE-1   ROUGE-L    METEOR     chrF2
count  4.000000  4.000000  4.000000  4.000000
mean   0.629752  0.356576  0.433488  0.598577
std    0.095040  0.075975  0.089868  0.117804
min    0.520373  0.281001  0.321001  0.451960
25%    0.584906  0.317291  0.395564  0.534903
50%    0.624376  0.342248  0.437358  0.609795
75%    0.669222  0.381533  0.475282  0.673468
max    0.749882  0.460806  0.538234  0.722759

Length Analysis:
Average Original True Length: 133.8 words
Average Original Predicted Length: 171.5 words
Average Normalized Length: 133.8 words


In [26]:
print("\nAverage Metrics:")
avg_metrics = eval_df[['ROUGE-1', 'ROUGE-L', 'METEOR', 'chrF2']].mean()

# Creating a DataFrame with average metrics
avg_metrics_df = pd.DataFrame(avg_metrics, columns=['Average Value']).reset_index()
avg_metrics_df.columns = ['Metric', 'Average Value']
display(avg_metrics_df)

print("\nEvaluation DataFrame:")
# Adjust display options
pd.set_option('display.max_colwidth', 200)

# Create a new DataFrame with shortened answers
eval_df_short = eval_df.copy()
eval_df_short['True Answer'] = eval_df_short['True Answer'].apply(lambda x: textwrap.shorten(x, width=200, placeholder='...'))
eval_df_short['Predicted Answer'] = eval_df_short['Predicted Answer'].apply(lambda x: textwrap.shorten(x, width=200, placeholder='...'))
display(eval_df[['Question', 'True Answer', 'Predicted Answer', 'ROUGE-1', 'ROUGE-L', 'METEOR', 'chrF2']].head())




Average Metrics:


,Metric,Average Value
0,ROUGE-1,0.629752
1,ROUGE-L,0.356576
2,METEOR,0.433488
3,chrF2,0.598577



Evaluation DataFrame:


,Question,True Answer,Predicted Answer,ROUGE-1,ROUGE-L,METEOR,chrF2
0,Is Loeys-Dietz syndrome inherited ?,"Loeys-Dietz syndrome is considered to have an autosomal dominant pattern of inheritance, which means one copy of the altered gene in each cell is sufficient to cause the disorder. In about 75 per...","Yes, Loeys-Dietz syndrome is inherited in an autosomal dominant pattern. This means that only one copy of the altered gene in each cell is sufficient to cause the disorder. While many individual...",0.642336,0.460806,0.454298,0.562551
1,What are the genetic changes related to cerebral cavernous malformation ?,"Mutations in at least three genes, KRIT1 (also known as CCM1), CCM2, and PDCD10 (also known as CCM3), cause familial cerebral cavernous malformations. The precise functions of these genes are not...","The genetic basis of cerebral cavernous malformations (CCMs) is complex and involves mutations in several genes, primarily three: _KRIT1_, _CCM2_, and _PDCD10_. These genes are involved in vario...",0.749882,0.329387,0.538234,0.722759
2,What is (are) lactate dehydrogenase deficiency ?,"Lactate dehydrogenase deficiency is a condition that affects how the body breaks down sugar to use as energy in cells, primarily muscle cells. There are two types of this condition: lactate dehyd...","Lactate dehydrogenase (LDH) deficiency is a rare genetic disorder affecting the body's ability to break down sugar for energy, primarily in muscle cells. There are two main types:\n\n* **Lactate ...",0.606417,0.355109,0.420418,0.657038
3,Is 21-hydroxylase deficiency inherited ?,How is 21-hydroxylase-deficient congenital adrenal hyperplasia passed through families? 21-hydroxylase-deficient congenital adrenal hyperplasia has an autosomal recessive pattern of inheritance. I...,"Yes, 21-hydroxylase deficiency is inherited in an autosomal recessive pattern. This means that a person needs to inherit two copies of a mutated gene, one from each parent, to have the condition....",0.520373,0.281001,0.321001,0.451960
